In [1]:


import win32com.client


# Connect to Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)
messages = inbox.Items

# Read emails and rewrite responses
for message in messages:
    subject = message.Subject
    body = message.Body
    print(subject)

ModuleNotFoundError: No module named 'win32com'